# 猫狗大战 - 迁移学习的一些尝试

## 1. 模型的训练方式

- 深度学习模型可以划分为 **训练** 和 **预测** 两个阶段。


- **训练** 又分为两种策略。一种是白手起家从头搭建模型进行训练，一种是通过预训练模型进行训练。


- **预测** 相对简单，直接用已经训练好的模型对数据集进行预测即可。（此时采用上面两种策略训练得到的模型都被视为“已训练模型”）

$$
\textbf{Application}
\begin{cases}
    \textbf{Training}
    \begin{cases}
        \color{Blue}{\textbf{Training From Scratch}} \\[2ex]
        \color{Green}{\textbf{Using Pre-trained Model}}
        \begin{cases}
            ^{\#} \textrm{1. Transfer Learning} \\[2ex]
            ^{\#} \textrm{2. Extract Feature Vector (Bottle Neck)} \\[2ex]
            ^{\#} \textrm{3. Fine-tune}
        \end{cases}
    \end{cases}
    \\[3ex]
    \textbf{Inference}: \textrm{Using Pre-trained Model}
\end{cases}
$$

## 2. 迁移学习的三种训练方式


$^{\#} \textbf{1. Transfer Learning}$：冻结预训练模型的全部卷积层，只训练自己定制的全连接层。


$^{\#} \textbf{2. Extract Feature Vector}$：先计算预训练模型的卷积层对所有训练和测试数据的特征向量，然后抛开预训练模型，只训练自己定制的简配版全连接网络。


$^{\#} \textbf{3. Fine-tune}$：冻结预训练模型的部分卷积层（通常是靠近输入的大部分卷积层），训练剩下的卷积层（通常是靠近输出的小部分卷积层）和全连接层。

> "Transfer Learning" 和 "Fine-tune" 其实没有严格的区分，只不过后者更常见于形容迁移学习的**后期微调**中。

## 3. 三种训练方式的对比

- 第一种和第二种训练得到的模型本质上并没有什么区别，但是第二种的计算复杂度要远远好于第一种。


- 第三种是对前两种方法的补充，以进一步提升模型性能。要注意的是，这种方法**并不一定能对模型有所提升**。


- 本质上来讲，这三种迁移学习的方式都是为了让预训练模型能够胜任新数据集的识别工作，能够让预训练模型原本的特征提取能力得到充分的释放和利用。但是，在此基础上如果想让模型能够达到更低的Loss，那么光靠迁移学习是不够的，靠的更多的还是**模型的结构以及新数据集的数量**。

------

# 开始实验

## 0. 数据集预处理

In [1]:
import os
import sys
import cv2
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from datetime import datetime
from tqdm import tqdm
from utils import get_params_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.applications import inception_v3, xception, resnet50, vgg16, vgg19
from keras.applications import InceptionV3, Xception, ResNet50, VGG16, VGG19
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
height = 299
labels = np.array([0] * 12500 + [1] * 12500)
train = np.zeros((25000, height, height, 3), dtype=np.uint8)
test = np.zeros((12500, height, height, 3), dtype=np.uint8)

for i in tqdm(range(12500)):
    img = cv2.imread('./train/cat/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i] = img[:, :, ::-1]
    
for i in tqdm(range(12500)):
    img = cv2.imread('./train/dog/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i + 12500] = img[:, :, ::-1]

for i in tqdm(range(12500)):
    img = cv2.imread('./test/%s.jpg' % str(i + 1))
    img = cv2.resize(img, (height, height))
    test[i] = img[:, :, ::-1]
    
print('Training Data Size = %.2f GB' % (sys.getsizeof(train)/1024**3))
print('Testing Data Size = %.2f GB' % (sys.getsizeof(test)/1024**3))

100%|█████████████████████████████████████████████████████| 12500/12500 [00:56<00:00, 222.85it/s]


Training Data Size = 6.24 GB
Testing Data Size = 3.12 GB


## 实验一：冻结全部卷积层 + 训练自己定制的全连接层

#### 要点

- **使用InceptionV3预训练模型**：该模型当初训练时的输入图像尺寸是299x299，且图像通道顺序为RGB，模型总参数为**两千多万个**。
- **预处理**：按照预训练模型原本的预处理方式对数据进行预处理。
- **基模型**：导入预训练模型，注意只导入卷积层部分，并锁定全部卷积层参数。
- **定制模型**：卷积层之后先接全局平均池化（GAP），再接Dropout，再接分类器，根据分类任务选择输出个数。模型可训练参数只有**两千个**。
- **优化器**：采用较小学习率的SGD。
- **数据准备**：将训练集划分为训练集和验证集。
- **定义回调函数以方便训练**：自动在每代结束保存模型，以val_loss作为监控指标进行早停，训练历史数据同步更新到Tensorboard供可视化。
- **需要用非常小的batch_size进行训练**：这样可以让模型更快更好的收敛。
- 可以看到，虽然卷积层全都已经锁定，但是由于样本依然需要从模型的输入一直计算到输出，因此训练还是比较耗时的，训练五代需要十几分钟。

In [7]:
# Preprocess: Standardization
x = Input(shape=(height, height, 3))
x = Lambda(inception_v3.preprocess_input)(x)

# Base Model: Freeze all conv layers
base_model = InceptionV3(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
for layer in base_model.layers:
    layer.trainable = False

# Customized Classifier
y = Dropout(0.2)(base_model.output)
y = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)

# Full Model: Pre-train Conv + Customized Classifier
model = Model(inputs=base_model.input, outputs=y, name='Transfer_Learning')
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model))

Trainable: 2049, Non-Trainable: 21802784


In [8]:
X_train, X_val, y_train, y_val = train_test_split(train, labels, shuffle=True, test_size=0.2, random_state=42)

In [9]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('transfer_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model.fit(x=X_train, y=y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val), callbacks=[es, mc, tb])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 163s - loss: 0.1887 - acc: 0.9367 - val_loss: 0.0853 - val_acc: 0.9798
Epoch 2/10
20000/20000 [==============================] - 161s - loss: 0.1220 - acc: 0.9566 - val_loss: 0.0700 - val_acc: 0.9818
Epoch 3/10
20000/20000 [==============================] - 161s - loss: 0.1084 - acc: 0.9604 - val_loss: 0.0601 - val_acc: 0.9834
Epoch 4/10
20000/20000 [==============================] - 161s - loss: 0.1086 - acc: 0.9602 - val_loss: 0.0599 - val_acc: 0.9824
Epoch 5/10
20000/20000 [==============================] - 161s - loss: 0.0991 - acc: 0.9630 - val_loss: 0.0565 - val_acc: 0.9822
Epoch 6/10
20000/20000 [==============================] - 161s - loss: 0.0984 - acc: 0.9626 - val_loss: 0.0514 - val_acc: 0.9840
Epoch 7/10
20000/20000 [==============================] - 160s - loss: 0.0925 - acc: 0.9649 - val_loss: 0.0509 - val_acc: 0.9840
Epoch 8/10
20000/20000 [========================

## 实验二：导出特征向量，再单独训练分类器

#### 要点

- **预处理**：导出训练集和测试集的特征向量之前，一定要记得按照预训练模型的要求进行预处理，否则导出的特征将不是模型的最佳表现。
- **基模型**：基模型同样由InceptionV3的卷积层部分以及全局平均池化（GAP）构成。
- **导出即预测**：所谓导出，其实就是让基模型直接对训练集和测试集进行预测，只不过预测出的不是类别，而是特征向量（特征图的浓缩版本）。
- **导出需要一定时间**：由于导出需要对数据集的所有图片进行预测，因此通常需要一两分钟才能完成。好的是，一旦完成，就可以完全和CNN拜拜了。
- **新模型的输入是特征向量**：新模型的输入不再是训练集的图像本身，而是经过预训练模型“消化”后的图像特征向量，该向量的第一个维度对应于每一个图像样本，其长度为样本的个数，第二个维度是基模型最后一层每个卷积核输出特征图的平均值，对于InceptionV3来说，第二个维度的长度是2048。
- **划分训练集和验证集**：为了训练时了解模型收敛情况，同样对特征向量划分训练集和验证集。
- **定制新模型**：由于已经导出特征向量，因此接下来只需训练一个输入特征长度为2048的全连接网络即可。
- 同样采用回调函数以及很小的batch_size（16）进行训练。可以看到训练快到飞起，5代训练仅用时十几秒钟，就可以达到0.02左右的val_loss。

In [3]:
# Preprocess: Standardization
x = Input(shape=(height, height, 3))
x = Lambda(inception_v3.preprocess_input)(x)

# Base Model: Extract feature vector of both train & test dataset
base_model = InceptionV3(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
train_gap = base_model.predict(train, batch_size=128)
test_gap = base_model.predict(test, batch_size=128)

X_train_gap, X_val_gap, y_train_gap, y_val_gap = train_test_split(train_gap, labels, shuffle=True, test_size=0.2, random_state=42)

In [4]:
# Input Shape: (Batch Size, Feature Vector length)
x = Input(shape=(X_train_gap.shape[1],))
y = Dropout(0.2)(x)
y = Dense(1, activation='sigmoid', kernel_initializer='he_normal', name='classifier')(y)
model_gap = Model(inputs=x, outputs=y, name='GAP')
model_gap.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_gap))

Trainable: 2049, Non-Trainable: 0


In [5]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('gap_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_gap.fit(x=X_train_gap, y=y_train_gap, batch_size=16, epochs=5, validation_data=(X_val_gap, y_val_gap), callbacks=[es, mc, tb])

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 3s - loss: 0.0754 - acc: 0.9856 - val_loss: 0.0317 - val_acc: 0.9912
Epoch 2/5
20000/20000 [==============================] - 3s - loss: 0.0270 - acc: 0.9926 - val_loss: 0.0261 - val_acc: 0.9918
Epoch 3/5
20000/20000 [==============================] - 3s - loss: 0.0219 - acc: 0.9936 - val_loss: 0.0241 - val_acc: 0.9922
Epoch 4/5
20000/20000 [==============================] - 3s - loss: 0.0199 - acc: 0.9941 - val_loss: 0.0234 - val_acc: 0.9924
Epoch 5/5
20000/20000 [==============================] - 3s - loss: 0.0191 - acc: 0.9940 - val_loss: 0.0229 - val_acc: 0.9920


## 实验三：尝试对模型进行微调，以进一步提升模型性能

### 1. Fine-tune所扮演的角色

拿到新数据集，想要用预训练模型处理的时候，通常大家都会先用上面实验一或者实验二里的方法看看预训练模型在新数据上的表现怎么样，摸个底。如果表现不错，还想看看能不能进一步提升，就可以试试Fine-tune（即解锁比较少的卷积层继续训练），但是不要期待会有什么质的飞跃。如果由于新数据集与原数据集（例如ImageNet数据集）的差别太大导致表现很糟，那么一方面可以考虑自己从头训练模型，另一方面也可以考虑解锁比较多层的训练，亦或干脆只用预训练模型的参数作为初始值，对模型进行完整训练。

### 2. Fine-tune的三种实现方式

其实基本思路都是一样的，就是解锁少数卷积层继续对模型进行训练。

- **场景1**：已经采用实验一里的方法，带着冻僵的卷积层训练好分类器了，现在想要进一步提升模型。

    - 实现方式：接着用实验一里的模型，直接解锁一小部分卷积层接着训练就成了，很简单。
    
    
- **场景2**：已经采用实验二里的方法，光把分类器训练好了，现在想要进一步提升模型。

    - 实现方式：重新搭一个预训练模型接新分类器，然后把实验二里训练好的分类器参数载入到新分类器里，解锁一小部分卷积层接着训练。
    
    
- **场景3**：刚上手，想要 Transfer Learning 和 Fine-tune 一气呵成。（这么做需要搭配很低的学习率，因此收敛可能会很慢）

    - 实现方式：和实验一里的操作一样，唯一不同的就是只冻僵一部分卷积层训练。

### 3. 具体实现 

下面我们接着实验二的脚步进一步对模型进行Fine-tune（即上面所说的场景2）。

#### 要点

- **基模型和定制模型**：构建和实验一里面完全相同的模型。要注意的是得把Dense层的名字定义的和实验二中一样（都叫'classifier'）。
- **导入分类器的参数**：由于实验二中已经将分类器（即Dense层）训练好了，因此这里只需要by_name的load_weight就可以让刚搭的模型满血复活了。
- **通过evaluate函数查看模型状态**：载入权重之前，模型的预测准确率很低，跟瞎猜（50%）差不多。而载入之后，准确率直接飙升到99%，说明权重载入的过程没有问题。

In [35]:
# Preprocess: Standardization
x = Input(shape=(height, height, 3))
x = Lambda(inception_v3.preprocess_input)(x)

# Base Model: Freeze all conv layers
base_model = InceptionV3(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
for layer in base_model.layers:
    layer.trainable = False

# Customized Classifier
y = Dropout(0.2)(base_model.output)
y = Dense(1, activation='sigmoid', kernel_initializer='he_normal', name='classifier')(y)

# Full Model: Pre-train Conv + Customized Classifier
model_finetune = Model(inputs=base_model.input, outputs=y, name='Fine-tuning')
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model_finetune.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_finetune))

# Raw Performance (Classifier not trained)
res = model_finetune.evaluate(X_train[:1000], y_train[:1000], verbose=0)
print("Loss: %.4f, Acc: %.2f%%" % (res[0], res[1] * 100))

Trainable: 2049, Non-Trainable: 21802784
Loss: 0.8281, Acc: 41.70%


In [36]:
# Load weights from previously trained Dense Layer.
weight_path = './gap_model_20171014_1137/DogVSCat-EP04-LOSS0.0229.h5'
model_finetune.load_weights(weight_path, by_name=True)

# Performance with Trained Classifier 
res = model_finetune.evaluate(X_train[:1000], y_train[:1000], verbose=0)
print("Loss: %.4f, Acc: %.2f%%" % (res[0], res[1] * 100))

Loss: 0.0142, Acc: 99.40%


![image](https://github.com/mtyylx/Resources/blob/master/CNN%20-%20Model%20InceptionV3.png?raw=true)
![image](https://github.com/mtyylx/Resources/blob/master/CNN%20-%20Inception%20Module.png?raw=true)

### 4. 如何选取要解锁的区域

- 如上图所示，由于我们使用的InceptionV3模型，是按照一个一个的Inception Module级联起来的，因此我们要选择Fine-tune的话应该也以Inception Module为最小单位（而不是单独的某一层）进行锁定和解锁。


- 通过看 model_finetune.summary() 可以基本确定上图中的 mixed10 区域（倒数34层）构成的 Inception Module 可以解锁训练下。

In [37]:
# Decide which layers to unlock
for layer in model_finetune.layers[-34:]:
    layer.trainable = True

In [38]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('finetune_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_finetune.fit(x=X_train, y=y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val), callbacks=[es, mc, tb])

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 186s - loss: 0.1276 - acc: 0.9520 - val_loss: 0.0650 - val_acc: 0.9798
Epoch 2/5
20000/20000 [==============================] - 161s - loss: 0.1095 - acc: 0.9587 - val_loss: 0.0637 - val_acc: 0.9812
Epoch 3/5
20000/20000 [==============================] - 162s - loss: 0.1060 - acc: 0.9587 - val_loss: 0.0546 - val_acc: 0.9830
Epoch 4/5
20000/20000 [==============================] - 161s - loss: 0.0983 - acc: 0.9623 - val_loss: 0.0566 - val_acc: 0.9824
Epoch 5/5
20000/20000 [==============================] - 162s - loss: 0.1033 - acc: 0.9611 - val_loss: 0.0522 - val_acc: 0.9830


> 可以看到，fine-tune后模型的Loss反而变差了，从实验二时的0.02上升到了0.05。这说明这个模型的能力也就到此为止了，无法再通过Fine-tune进行提高。如果还想提高的话，只能往两个方向努力：

> - 修改模型结构（包括融合更多模型）

> - 获得更多数据（包括图像增强等方法）